In [13]:
from DocumentDBVectorOperations import DocumentDBVectorSearch
from Chunking import chunking, combine_text
from DocumentLoader import load_and_read_pdf
from GenerateEmbeddings import generate_embeddings

In [2]:
docdb_obj =  DocumentDBVectorSearch()

In [4]:
pages = load_and_read_pdf(file_path = "VisionTransformers.pdf")

final_text = combine_text(pages)

chunks_of_text = chunking(final_text)

In [5]:
docdb_obj.add_texts(chunks_of_text)

Documents inserted successfully.


In [22]:
prompt = "What is Vision Transformer"
prompt_embed = generate_embeddings([query])[0].tolist()

In [23]:
docdb_obj.collection.create_index ([("embedding","vector")], 
    vectorOptions= {
        "type": "hnsw", 
        "similarity": "cosine",
        "dimensions": 1024,
        "m": 16,
        "efConstruction": 64},
    name="my_vss_index")

'my_vss_index'

In [ ]:
# projection = {
# "_id":0,
# "title": 1, 
# "overview": 1}

# #Semantic search function
# def search_semantic(embedding):
#     query = {"vectorSearch" : {"vector" : embedding, "path": "embedding", "similarity": "cosine", "k": 3}}
#     results = docdb_obj.collection.aggregate([{'$search': query},{"$project": projection}])
#     return list(results)

# #Text search function
# def search_text(keyword):
#     results = collection.aggregate([{"$match": {"$text": {"$search": keyword}}},{"$project": projection},{"$limit": 3}])
#     return list(results)

# #Hybrid query function
# def search_hybrid(keyword):
#     results1 = search_semantic(keyword)[:2]
#     results2 = search_text(keyword)[:2]
#     combined_results = results1 + results2
#     combined_results_as_tuples = [tuple(d.items()) for d in combined_results]
#     union_result = list(set(combined_results_as_tuples))
#     return union_result[:3]

In [24]:
# projection = {
# "_id":0,
# "title": 1, 
# "overview": 1}

#Semantic search function
def search_semantic(embedding):
    query = {"vectorSearch" : {"vector" : embedding, "path": "embedding", "similarity": "cosine", "k": 3}}
    results = docdb_obj.collection.aggregate([{'$search': query}])
    return list(results)

#Text search function
def search_text(keyword):
    results = docdb_obj.collection.aggregate([{"$match": {"$text": {"$search": keyword}}},{"$limit": 3}])
    return list(results)

#Hybrid query function
def search_hybrid(keyword):
    results1 = search_semantic(keyword)[:2]
    results2 = search_text(keyword)[:2]
    combined_results = results1 + results2
    combined_results_as_tuples = [tuple(d.items()) for d in combined_results]
    union_result = list(set(combined_results_as_tuples))
    return union_result[:3]

In [27]:
results = search_semantic(prompt_embed)

In [28]:
final_text = ""
for result in results:

    final_text += result['text']


In [29]:
final_text

'Transformers appear not to saturate within the range tried, motivating future scaling efforts.\n4.5 I NSPECTING VISION TRANSFORMER\nInput\n Attention\nFigure 6: Representative ex-\namples of attention from the\noutput token to the input\nspace. See Appendix D.7 for\ndetails.\nTo begin to understand how the Vision Transformer processes im-\nage data, we analyze its internal representations. The ﬁrst layer of\nthe Vision Transformer linearly projects the ﬂattened patches into a\nlower-dimensional space (Eq. 1). Figure 7 (left) shows the top prin-\ncipal components of the the learned embedding ﬁlters. The com-\nponents resemble plausible basis functions for a low-dimensional\nrepresentation of the ﬁne structure within each patch.\nAfter the projection, a learned position embedding is added to the\npatch representations. Figure 7 (center) shows that the model learns\nto encode distance within the image in the similarity of position em-\nbeddings, i.e. closer patches tend to have more simi